# 微分可能ビリーフロパゲーション復号法

Copyright (c) 2022 Tadashi Wadayama  
Released under the MIT license  
https://opensource.org/licenses/mit-license.php

In [1]:
using LinearAlgebra

## 検査行列定義ファイル(alist形式)

検査行列定義ファイルは、alist形式を利用します。
alist形式の定義は

http://www.inference.org.uk/mackay/codes/alist.html

を参照してください。

定義ファイル類は下記のサイトを御覧ください。

http://www.inference.org.uk/mackay/codes/data.html

https://www.uni-kl.de/channel-codes/ml-simulation-results/

In [2]:
filename = "6.3.alist"

"6.3.alist"

## Alist ファイルの読み込み

In [3]:
fp = open(filename, "r") 
dict = Dict()
rdict = Dict()

line = readline(fp)
u = split(line, " ") 
n = parse(Int64,u[1])
m = parse(Int64,u[2])
println("n = ", n) 
println("m = ", m)
line = readline(fp)
u = split(line, " ") 
cmax = parse(Int64,u[1])
rmax = parse(Int64,u[2]) 
println("cmax = ", cmax) 
println("rmax = ", rmax)
line = readline(fp)
line = readline(fp)
id = 1
for i in 1:n
    line = readline(fp) 
    u = split(line, " ") 
    for j in 1:cmax
        v = parse(Int64, u[j])
        if v != 0
            dict[(v,i)] = id
            rdict[id] = (v,i)
            id += 1         
        end
    end
end
esize = length(dict)

H = zeros(Int, m,n)
for x in keys(dict)
    H[x[1], x[2]] = 1
end

U = zeros(Int, n, esize)
for edge in dict
    j = edge[2]
    i = edge[1][2]
    U[i,j]=1
end

V = zeros(Int, m, esize)
for edge in dict
    j = edge[2]
    i = edge[1][1]
    V[i,j]=1
end

n = 6
m = 3
cmax = 2
rmax = 3


### 微分可能ビリーフプロパゲーション復号法

下記の実装は、数値計算誤差に弱いので反復計算には向きません。
数値誤差対策については、BP-train.ipynbに利用されている実装を参考にしてください。

In [4]:
var = 0.794328
lambda = 2.0*[1.620803, 0.264281, -0.031637, 
    -0.127654, 0.746347, 1.003543]/var
alpha = zeros(esize) 
beta = zeros(esize)
bmod(x) = x-2*floor(x/2)
for i in 1:3
    beta = (U'*U-I)*alpha + U'*lambda
    tmp = exp.((V'*V-I)*log.(abs.(tanh.(beta/2))))
    alpha_abs = 2*atanh.(tmp)
    tmp = 1 .- 2*V'*bmod.(V*((-sign.(beta).+ 1)/2))
    alpha_sign = tmp .* sign.(beta)
    alpha = alpha_sign .* alpha_abs 
end
    gamma = U*alpha + lambda

6-element Vector{Float64}:
 4.39741886230254
 1.6925246559341063
 1.7110998961710588
 1.7110998961710584
 2.083964089240627
 2.703277039402432

### 対数事後確率比

上記の設定において、対数事後確率比をBP復号法により計算すると

4.397419
1.692525
1.711101
1.711101
2.083965
2.703277

となることが知られている(和田山著「誤り訂正技術の基礎」演習問題14.4)。
上記の計算結果は十分な精度で一致している。